# Syntax and the Corresponding Magic

In [ ]:
class Base:
    def __init__(self, name):
        self.name = name

In [ ]:
b = Base('NaOH')

In [ ]:
b

In [ ]:
print(repr(b))

```
















```

## Stringy Types

In [ ]:
class Stringy:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.name!r})"
    
    def __str__(self) -> str:
        return f"It's name...is {self.name}"

In [ ]:
a = Stringy('apple')
a

In [ ]:
repr(a)

In [ ]:
str(a)

In [ ]:
class StringyToo:
    def __init__(self, name):
        self.name = name
        
    def to_string(self):
        return f"It's name...is {self.name}"

In [ ]:
b = StringyToo('cheese')
b

In [ ]:
b.to_string()

```
















```

### Quacks like...

String-like objects can be used in place of other strings

In [ ]:
f"I have an object. {b}."

In [ ]:
f"I have an object. {a}. It was made with {a!r}"

```
















```

## Addable Types

Implementing the mathematical magic methods can let you use any of the infix operators to manipulate your objects as makes sense.

    x + y      x ** y      x & y      x << y
    x - y      x // y      x | y      x >> y
    x * y      x % y       x ^ y
    x / y      x @ y       

In [ ]:
class Mathy(Stringy):
    def __add__(self, other) -> str:
        return f"{self.name} and {other.name}"

In [ ]:
Mathy("coke") + Mathy("mentos")

In [ ]:
Mathy("eggs") + Mathy("spam") + Mathy("cheese")

In [ ]:
class Addy(Stringy):
    def __add__(self, other) -> 'Addy':
        return Addy(f"{self.name} and {other.name}")

In [ ]:
Addy("eggs") + Addy("spam") + Addy("cheese")

```
















```

### Builtins that Add

The built-in `sum()` uses `+` to add objects together. It uses a start value to 

In [ ]:
help(sum)

In [ ]:
sum([Addy("eggs"), Addy("spam"), Addy("cheese"), Addy('funny walks')])

In [ ]:
class Addy2(Stringy):
    def __add__(self, other) -> 'Addy2':
        return Addy2(f"{self.name} and {other.name}")
    
    def __radd__(self, other) -> 'Addy2':
        if not other:
            return self

In [ ]:
0 + Addy2('just me')

In [ ]:
sum([Addy2("eggs"), Addy2("spam"), Addy2("cheese"), Addy2('funny walks')])

```
















```

## Orderable Types

In [ ]:
doctors = [
    Stringy('Eccleston'), 
    Stringy('Tennant'), 
    Stringy('Smith'), 
    Stringy('Capaldi'), 
    Stringy('Whittaker'),
]

How can we sort these? We could pass a `key` argument into `sorted()`:

In [ ]:
sorted(doctors, key=lambda i: i.name)

```
















```

In [ ]:
class Comparable(Stringy):
    def __lt__(self, other):
        return self.name < other.name

In [ ]:
Comparable('apple') < Comparable('banana')

In [ ]:
Comparable('pear') < Comparable('banana')

In [ ]:
doctors = [Comparable(d.name) for d in doctors]
doctors

In [ ]:
sorted(doctors)

In [ ]:
max(doctors)

In [ ]:
min(doctors)

Also, not everything can take a `key` argument:

In [ ]:
import heapq

docheap = doctors[:]
heapq.heapify(docheap)

In [ ]:
while docheap:
    print(heapq.heappop(docheap))

```
















```

## Container Types

In [ ]:
import re

import requests


def get_xkcd_hotlink(comic_number):
    response = requests.get(f"https://xkcd.com/{comic_number}/")
    match = re.search(
        r"Image URL \(for hotlinking/embedding\):\s*(?P<url>http.*(jpg|png))", 
        response.text,
    )
    if not match:
        return None

    src = match.groupdict()["url"]
    return src        


class XKCDContainer:        
    def __getitem__(self, key):
        url = get_xkcd_hotlink(key)
        if url is None:
            raise KeyError("couldn't find XKCD comic with that number")
        return url
        
    def __contains__(self, key):
        return get_xkcd_hotlink(key) is not None

In [ ]:
xkcd_container = XKCDContainer()

In [ ]:
xkcd_container[364]

In [ ]:
1000 in xkcd_container

In [ ]:
10000 in xkcd_container

```
















```

## Proprietary Magic

https://ipython.readthedocs.io/en/stable/config/integrating.html#rich-display

IPython notebook can display richer representations of objects. To use this, you can define any of a number of `_repr_*_()` methods. Note that these are surrounded by single, not double underscores.

The notebook can display `svg`, `png`, `jpeg`, `html`, `javascript`, `markdown` and `latex`. If the methods don’t exist, or return `None`, it falls back to a standard `repr()`.

In [ ]:
class XKCDComic:
    def __init__(self, url):
        self.url = url

    def _repr_html_(self):
        return f"<img src='{self.url}'>"


class XKCDViewer(XKCDContainer):
    def __getitem__(self, key):
        return XKCDComic(super().__getitem__(key))

In [ ]:
xkcd = XKCDViewer()

In [ ]:
xkcd[843]